from  https://design.ros2.org/articles/clock_and_time.html


> To provide a simplified time interface we will provide a ROS time and duration datatype. To query for the latest time a ROS Clock interface will be provided. A TimeSource can manage one or more Clock instances.




## ROS clock (Interface) (?)

suppose you want to have a node that is managing time, this is maybe for all internal ros node through this clock mechanism, but maybe also getting some interface from the outside??!
is it really going to work that way? how the time is managed in ros?

>An important aspect of using an abstracted time is to be able to manipulate time. In some cases, speeding up, slowing down, or pausing time entirely is important for debugging.

> The ability to support pausing time requires that we not assume that the time values are always increasing.

> When communicating the changes in time propagation, the latencies in the communication network becomes a challenge. Any change in the time abstraction must be communicated to the other nodes in the graph, but will be subject to normal network communication latency. This inaccuracy is proportional to the latency of communications and also proportional to the increase in the rate at which simulated time advances compared to real time

> The final challenge is that the time abstraction must be able to jump backwards in time, a feature that is useful for log file playback. This behavior is similar to a system clock after a negative date change, and requires developers using the time abstraction to make sure their algorithm can deal with the discontinuity. Appropriate APIs must be provided for to the developer API to enable notifications of jumps in time, both forward and backwards.





SystemTime will be directly tied to the system clock.

The ROSTime will report the same as SystemTime when a ROS Time Source is not active. 

When the ROS time source is active ROSTime will return the latest value reported by the Time Source.
ROSTime is considered active when the parameter use_sim_time is set on the node.

(what is the use_sim_time parameter?/)







## ROS Time Source
Default Time Source
To implement the time abstraction the following approach will be used.

The time abstraction can be published by one source on the /clock topic. 
The topic will contain the most up to date time for the ROS system. 
If a publisher exists for the topic, it will override the system time when using the ROS time abstraction.
If /clock is being published, calls to the ROS time abstraction will return the latest time received from the /clock topic. 
If time has not been set it will return zero if nothing has been received. 
A time value of zero should be considered an error meaning that time is uninitialized.

The frequency of publishing the /clock as well as the granularity are not specified as they are application specific.

It is possible that the user may have access to an out of band time source which can provide better performance than the default source the /clock topic. It might be possible that for their use case a more advanced algorithm would be needed to propagate the simulated time with adequate precision or latency with restricted bandwidth or connectivity. The user will be able to switch out the time source for the instance of their Time object as well as have the ability to override the default for the process.

For the current implementation a TimeSource API will be defined such that it can be overridden in code. If in the future a common implementation is found that would be generally useful it could be extended to optionally dynamically select the alternative TimeSource via a parameter similar to enabling the simulated time.

The SystemTime, SteadyTime, and ROSTime API’s will be provided by each client library in an idiomatic way, but they may share a common implementation, e.g. provided by rcl. However, if a client library chooses to not use the shared implementation then it must implement the functionality itself.


The implementation from client library will provide Time, Duration, and Rate datatypes, for all three time source abstractions.

The Clock will support a sleep_for function as well as a sleep_until method using a Duration or Time argument respectively. The implementation will also provide a Timer object which will provide periodic callback functionality for all the abstractions.




## My Time Modeling Excercise

okay what do I want to do here.


I want to create a publisher for custom simulated time.
and also a subscriber.

maybe for each of them I would need some API, or additional messages.


but for first try I would need a publisher that just publish the system time.



so finally after many try and errors, I got it done. The thing is the format of time bublished is not quite what I like!
what about the client?

```
[INFO] [1614494219.541160400] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494219, nanosec=539981700)"
[INFO] [1614494220.039608900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494220, nanosec=37652600)"
[INFO] [1614494220.538712600] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494220, nanosec=537591700)"
[INFO] [1614494221.037867600] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494221, nanosec=36693900)"
[INFO] [1614494221.538555200] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494221, nanosec=537445200)"
[INFO] [1614494222.041329400] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494222, nanosec=38853300)"
[INFO] [1614494222.542033400] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494222, nanosec=540807200)"
[INFO] [1614494223.040536100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494223, nanosec=39470300)"
[INFO] [1614494223.539639900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494223, nanosec=538520900)"
[INFO] [1614494224.039060900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494224, nanosec=38018200)"
[INFO] [1614494224.542671700] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494224, nanosec=541514900)"
[INFO] [1614494225.036846800] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494225, nanosec=36040500)"
[INFO] [1614494225.541828100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494225, nanosec=540721700)"
[INFO] [1614494226.038097500] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494226, nanosec=36952400)"
[INFO] [1614494226.537827400] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494226, nanosec=536878200)"
[INFO] [1614494227.038622800] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494227, nanosec=37472000)"
[INFO] [1614494227.540873700] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494227, nanosec=539771700)"
[INFO] [1614494228.039700300] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494228, nanosec=38659500)"
[INFO] [1614494228.541483100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494228, nanosec=540143100)"
[INFO] [1614494229.038634700] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494229, nanosec=37497600)"
[INFO] [1614494229.539185900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494229, nanosec=537969900)"
[INFO] [1614494230.039896900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494230, nanosec=38824900)"
[INFO] [1614494230.542178300] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494230, nanosec=541003400)"
[INFO] [1614494231.040638500] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494231, nanosec=39495600)"
[INFO] [1614494231.541539800] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494231, nanosec=540414600)"
[INFO] [1614494232.037726900] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494232, nanosec=36668600)"
[INFO] [1614494232.541264800] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494232, nanosec=540326500)"
[INFO] [1614494233.040571100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494233, nanosec=39525900)"
[INFO] [1614494233.537810100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494233, nanosec=536627700)"
[INFO] [1614494234.041588100] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494234, nanosec=40460400)"
[INFO] [1614494234.539564500] [time_publisher]: Publishing Time for : "builtin_interfaces.msg.Time(sec=1614494234, nanosec=538433200)"
^CTraceback (most recent call last):